# Compute Fuzzy Accuracies
### Sarah M. McDonald, smcdonald@chesapeakebay.net

Imports and paths

In [5]:
# imports
import pandas as pd
import geopandas as gpd

# paths
folder = r"C:\Users\smcdonald\Documents\Data\LULCC_2024ed\AccuracyAssessment\clean_points"
points_path = f"{folder}/lcc_aa_points_cleaned.gpkg"
ta_lc  = f"{folder}/fuzzy_TA/DC_lc_TA_3x3.dbf"
ta_lcc = f"{folder}/fuzzy_TA/DC_lcc_TA_3x3.dbf"
cw_path = r"C:\Users\smcdonald\Documents\Data\LULCC_2024ed\AccuracyAssessment\CIC\t1-t3_lc_change_values KEY.csv"

Read in Data

In [2]:
lc = (
    gpd.read_file(ta_lc)
    .drop('geometry', axis=1)
)
lcc = (
    gpd.read_file(ta_lcc)
    .drop('geometry', axis=1)
)
points = (
    gpd.read_file(points_path)
    .drop('geometry', axis=1)
)
print(lc.columns)
print(lcc.columns)
print(points.columns)

Index(['UID', 'VALUE_1', 'VALUE_2', 'VALUE_3', 'VALUE_4', 'VALUE_5', 'VALUE_6',
       'VALUE_7', 'VALUE_8', 'VALUE_9', 'VALUE_10', 'VALUE_11', 'VALUE_12'],
      dtype='object')
Index(['UID', 'VALUE_13', 'VALUE_15', 'VALUE_16', 'VALUE_17', 'VALUE_18',
       'VALUE_19', 'VALUE_23', 'VALUE_31', 'VALUE_35', 'VALUE_36', 'VALUE_37',
       'VALUE_38', 'VALUE_39', 'VALUE_43', 'VALUE_45', 'VALUE_46', 'VALUE_47',
       'VALUE_48', 'VALUE_51', 'VALUE_53', 'VALUE_54', 'VALUE_56', 'VALUE_57',
       'VALUE_58', 'VALUE_59', 'VALUE_61', 'VALUE_63', 'VALUE_65', 'VALUE_67',
       'VALUE_68', 'VALUE_69', 'VALUE_73', 'VALUE_75', 'VALUE_76', 'VALUE_78',
       'VALUE_79', 'VALUE_81', 'VALUE_83', 'VALUE_85', 'VALUE_86', 'VALUE_87',
       'VALUE_89', 'VALUE_93', 'VALUE_95', 'VALUE_96', 'VALUE_97', 'VALUE_98',
       'VALUE_105', 'VALUE_107', 'VALUE_108', 'VALUE_115', 'VALUE_117',
       'VALUE_118', 'VALUE_125', 'VALUE_128', 'VALUE_129', 'VALUE_210',
       'VALUE_211', 'VALUE_212', 'VALUE_1003', 'VA

Merge Ground Truth to TA Data

In [30]:
lc_df = (
    lc.merge((
            points
            .filter(items=['uid', 'type', 'GrndTruth', 'validGT'])
        )
        , left_on='UID', right_on='uid')
)

lcc_df = (
    lcc.merge((
            points
            .filter(items=['uid', 'type', 'GrndTruth', 'validGT'])
        )
        , left_on='UID', right_on='uid')
)

Add the LC Raster Value for Easy Querying

In [31]:
cw = pd.read_csv(cw_path)
cw.loc[:, 'static'] = cw['class'].str.split(' to ', n=1, expand=True)[1]
cw.loc[cw['static'].isna(), 'static'] = cw['class']
cw = (
    cw
    .filter(items=['value','static'], axis=1)
    .merge((
        cw
        .filter(items=['value', 'class'], axis=1)
    ), 
    left_on='static', right_on='class')
    .rename(columns={'value_x':'value', 'value_y':'static_value'})
    .filter(items=['value', 'static_value'], axis=1)
)

lc_df = (
    lc_df
    .merge(cw, left_on='GrndTruth', right_on='value')
    .drop('value', axis=1)
)

Compute Boolean Column showing if the GrndTruth Value Exists within the Window

In [ ]:
# iterate unique Ground Truth values and mark as T/F if any area for that class is present
